In [1]:
import torch
import torch.nn as nn
import fastai.vision.all as fv

In [2]:
def load_data(folder, img_size,batch_size):
    tmfs = fv.aug_transforms(flip_vert=False,
                            max_rotate=10,
                            max_lighting=0.25,
                            max_zoom=1.2,
                            max_warp=0.2)
    data = fv.DataBlock(blocks  = (fv.ImageBlock, fv.CategoryBlock),
                        get_items = fv.get_image_files,
                        get_y     = fv.parent_label, #Como a partir de la imagen encpontarr la categoria
                        splitter  = fv.GrandparentSplitter(), #si el abyelo es train o test
                        item_tfms = fv.Resize(img_size),#transformcaiones antes de juntar en batch y despues de juntar en batch)
                        batch_tfms = tmfs)
    return data.dataloaders(folder, bs=batch_size)#Agarra las imagenes y las regresa en batches

birds = load_data("/storage/birds", img_size=128, batch_size=64)

In [3]:
##Flatten es una capa que ayuda a pasar de la parte convolucional a la parte lineal

#Lo que hace la clase es que le pasamos algo y nos regresa eso mismo pero sin los 1
class Flatten(nn.Module):   #Creamos un módulo que hereda de nn.Module
    def __init__(self):    #Para crear un objeto
        super().__init__() #Le decimos a nn.Module que se inicie
    
    def forward( self, x ): #
        #return x.squeeze()
        return x.reshape (x.shape[0],-1) #Toma la primera y adivina el resto
        #Se agrega esta porque si el batch es de tam 1, squezze lo elimina
        

In [4]:
lx97  = nn.Sequential( #Sequential para cosas sencillas
    nn.BatchNorm2d( 3 ), #Ideal comenzar normalizando 
    #Convolucion de entrada 3, de 32 filtros, de tamaño 3x3, 
    #sride -> ¿cuánto brinca?, padding -> pone el brde negro para no perder pixeles
    nn.Conv2d( 3, 32, kernel_size = 3, stride = 2, padding = 1),
    
    #Acivacion
    nn.ReLU(),
    ##ResBlock( crear_residual( 16 ) ),
    #Reducir el tamaño de a  imagen (será más rápido)
    #Cada cuadro de 2x2 agarra el más grande.
    #Reduce a la mitad
    #nn.MaxPool2d(2),
    #nn.Conv2d( 16, 32, kernel_size=2 ),
    #nn.ReLU(),
    nn.BatchNorm2d( 32 ),
    nn.Conv2d( 32, 32, kernel_size=2, stride = 1, padding = 1 ),
    nn.ReLU(),
    nn.BatchNorm2d( 32 ),
    
    #Empezamos en la cantidad de filtros con los que acabamos en Conv2d anterior
    nn.Conv2d( 32, 64, kernel_size = 2, stride = 1, padding = 1),
    
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d( 64, 128, kernel_size = 3, stride = 1, padding = 1),
    nn.ReLU(),
    nn.BatchNorm2d( 128 ),
    nn.MaxPool2d(2),
    nn.Conv2d( 128, 256, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 256 ),
    nn.ReLU(),
    nn.Conv2d( 256, 256, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 256 ),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d( 256, 512, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 512 ),
    nn.ReLU(),
    #--Aquí acabamos con la parte convolucional--#
    
    #--Llegamos a la parte lineal--#
    
    nn.AdaptiveAvgPool2d(1), #Reducir el tamaño a 1x1xn_canales
    #Podemos usar diferentes métodos para llegar a 1x1
    Flatten(), #Usamos la funcion que habiamos creado pra eliminar las de dim 1
    #Los quita para poder usar Linear
    nn.Linear( 512, 256), #Completamente conectada
    nn.ReLU(),
    nn.Linear( 256, birds.c) #salida con el total de clases de los datos
)

In [5]:
learn_birds = fv.Learner( birds, lx97, opt_func=fv.ranger, metrics=fv.accuracy).to_fp16()

In [ ]:
learn_birds.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))

Modifica más las últimas capas en comparación con las primeras (que ya están pre-entrenadas) para no arruinar nuestro modelo.

Cada modelo está partido. Lo que hace es dividir el rango entre los pedazos y entrenar cada uno de ellos con un lr diferente

Si proponemos nuestros modelos, debemos decir cómo partirlo (para entrenar con slices)

### ¿Cómo partir nuestro modelo?

Al crear el Learner le debemos indocar el parámetro splitter, que regresa los pedazos del modelo

In [7]:
lx97

Sequential(
  (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): ReLU()
  (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(256, eps=1e-05, momentum=0.1

In [10]:
def my_own_splitter(model): #Tú eliges cómo dividirlos. La última parte (lineal), debe tener su propio grupo
    return [list(group.parameters()) for group in [model[0:2],model[2:8],model[8:10],model[10:15],model[15:25],model[25:]]]

In [12]:
learn = fv.Learner(birds, lx97,
                  opt_func=fv.ranger, metrics=fv.accuracy,
                  splitter=my_own_splitter)

In [14]:
learn.freeze_to(-1) #Congela todo excepto laúltima

In [15]:
learn.summary()

Sequential (Input shape: ['64 x 3 x 128 x 128'])
Layer (type)         Output Shape         Param #    Trainable 
BatchNorm2d          64 x 3 x 128 x 128   6          True      
________________________________________________________________
Conv2d               64 x 32 x 64 x 64    896        False     
________________________________________________________________
ReLU                 64 x 32 x 64 x 64    0          False     
________________________________________________________________
BatchNorm2d          64 x 32 x 64 x 64    64         True      
________________________________________________________________
Conv2d               64 x 32 x 65 x 65    4,128      False     
________________________________________________________________
ReLU                 64 x 32 x 65 x 65    0          False     
________________________________________________________________
BatchNorm2d          64 x 32 x 65 x 65    64         True      
_________________________________________________

Ahora vamos a entrenar el modelo a pedazos.

(Con las primeras capas congeladas)

In [16]:
learn.fit_one_cycle(1,slice(1e-6, 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.302774,5.290855,0.005523,00:14
